In [53]:
import torch
from torch import nn, optim
import numpy as np
from torch.utils.data import DataLoader
import pandas as pd
import ast

from collections import Counter
from sklearn.model_selection import train_test_split


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)

cuda:0


In [48]:
class Model(nn.Module):
    def __init__(self, dataset):
        super(Model, self).__init__()
        self.lstm_size = 128
        self.embedding_dim = 128
        self.num_layers = 3

        n_vocab = len(dataset.uniq_words)
        self.embedding = nn.Embedding(
            num_embeddings=n_vocab,
            embedding_dim=self.embedding_dim,
        )
        self.lstm = nn.LSTM(
            input_size=self.lstm_size,
            hidden_size=self.lstm_size,
            num_layers=self.num_layers,
            dropout=0.2,
        )
        self.fc = nn.Linear(self.lstm_size, n_vocab)

    def forward(self, x, prev_state):
        embed = self.embedding(x)
        output, state = self.lstm(embed, prev_state)
        logits = self.fc(output)
        return logits, state

    def init_state(self, sequence_length):
        return (torch.zeros(self.num_layers, sequence_length, self.lstm_size),
                torch.zeros(self.num_layers, sequence_length, self.lstm_size))

In [42]:
class Dataset(torch.utils.data.Dataset):
    def __init__(
        self,
        train_df
    ):
        self.words = self.load_words()
        self.uniq_words = self.get_uniq_words()

        self.index_to_word = {index: word for index, word in enumerate(self.uniq_words)}
        self.word_to_index = {word: index for index, word in enumerate(self.uniq_words)}

        self.words_indexes = [self.word_to_index[w] for w in self.words]

    def load_words(self):
        text = train_df.str.cat(sep=' ')
        return text.split(' ')

    def get_uniq_words(self):
        word_counts = Counter(self.words)
        return sorted(word_counts, key=word_counts.get, reverse=True)

    def __len__(self):
        return len(self.words_indexes) - 4

    def __getitem__(self, index):
        return (
            torch.tensor(self.words_indexes[index:index+4]),
            torch.tensor(self.words_indexes[index+1:index+4+1]),
        )

In [58]:
def train(dataset, model, results_list, batch_s=256, max_epochs=10, sequence_length=4):
    model.train()

    dataloader = DataLoader(dataset, batch_size=batch_s)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(max_epochs):
        state_h, state_c = model.init_state(sequence_length)

        for batch, (x, y) in enumerate(dataloader):
            optimizer.zero_grad()

            y_pred, (state_h, state_c) = model(x, (state_h, state_c))
            loss = criterion(y_pred.transpose(1, 2), y)

            state_h = state_h.detach()
            state_c = state_c.detach()

            loss.backward()
            optimizer.step()

            print({ 'epoch': epoch, 'batch': batch, 'loss': loss.item() })
            results_list.append([epoch, batch, loss.item()])

In [50]:
def predict(dataset, model, text, next_words=100):
    model.eval()

    words = text.split(' ')
    state_h, state_c = model.init_state(len(words))

    for i in range(0, next_words):
        x = torch.tensor([[dataset.word_to_index[w] for w in words[i:]]])
        y_pred, (state_h, state_c) = model(x, (state_h, state_c))

        last_word_logits = y_pred[0][-1]
        p = torch.nn.functional.softmax(last_word_logits, dim=0).detach().numpy()
        word_index = np.random.choice(len(last_word_logits), p=p)
        words.append(dataset.index_to_word[word_index])

    return words

In [51]:
df = pd.read_csv('data/RAW_recipes.csv')
recipe_df = df[["steps"]]

recipe_df["processed"] = [". ".join(ast.literal_eval(step_array)) for step_array in recipe_df.steps]

train_df, test_df = train_test_split(recipe_df["processed"], train_size=.8, test_size=.2)

/home/jmdanie/Documents/Code/Hackathons/Bon-Hacketit/venv/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [59]:
dataset = Dataset(train_df)
model = Model(dataset)
results = []

train(dataset, model, results)
print(predict(dataset, model, text='Add '))

{'epoch': 0, 'batch': 0, 'loss': 11.242292404174805}
{'epoch': 0, 'batch': 1, 'loss': 11.238499641418457}
{'epoch': 0, 'batch': 2, 'loss': 11.226659774780273}
{'epoch': 0, 'batch': 3, 'loss': 11.217924118041992}
{'epoch': 0, 'batch': 4, 'loss': 11.214835166931152}
{'epoch': 0, 'batch': 5, 'loss': 11.191398620605469}
{'epoch': 0, 'batch': 6, 'loss': 11.167433738708496}
{'epoch': 0, 'batch': 7, 'loss': 11.154248237609863}
{'epoch': 0, 'batch': 8, 'loss': 11.054868698120117}
{'epoch': 0, 'batch': 9, 'loss': 10.949844360351562}
{'epoch': 0, 'batch': 10, 'loss': 10.803844451904297}
{'epoch': 0, 'batch': 11, 'loss': 10.574508666992188}
{'epoch': 0, 'batch': 12, 'loss': 10.424949645996094}
{'epoch': 0, 'batch': 13, 'loss': 10.010403633117676}
{'epoch': 0, 'batch': 14, 'loss': 9.888875961303711}
{'epoch': 0, 'batch': 15, 'loss': 9.780108451843262}
{'epoch': 0, 'batch': 16, 'loss': 9.644513130187988}
{'epoch': 0, 'batch': 17, 'loss': 9.052526473999023}
{'epoch': 0, 'batch': 18, 'loss': 8.927183

{'epoch': 0, 'batch': 154, 'loss': 7.084575653076172}
{'epoch': 0, 'batch': 155, 'loss': 6.848651885986328}
{'epoch': 0, 'batch': 156, 'loss': 6.781240463256836}
{'epoch': 0, 'batch': 157, 'loss': 6.283814907073975}
{'epoch': 0, 'batch': 158, 'loss': 6.34154748916626}
{'epoch': 0, 'batch': 159, 'loss': 7.044858932495117}
{'epoch': 0, 'batch': 160, 'loss': 6.322938442230225}
{'epoch': 0, 'batch': 161, 'loss': 6.229717254638672}
{'epoch': 0, 'batch': 162, 'loss': 6.4532999992370605}
{'epoch': 0, 'batch': 163, 'loss': 6.341999530792236}
{'epoch': 0, 'batch': 164, 'loss': 6.28667688369751}
{'epoch': 0, 'batch': 165, 'loss': 6.280598163604736}
{'epoch': 0, 'batch': 166, 'loss': 5.97722864151001}
{'epoch': 0, 'batch': 167, 'loss': 6.877725601196289}
{'epoch': 0, 'batch': 168, 'loss': 6.231250762939453}
{'epoch': 0, 'batch': 169, 'loss': 6.182936191558838}
{'epoch': 0, 'batch': 170, 'loss': 6.239219665527344}
{'epoch': 0, 'batch': 171, 'loss': 6.64828634262085}
{'epoch': 0, 'batch': 172, 'los

KeyboardInterrupt: 